In [1]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd

In [2]:
# webpage variable set for scraping

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
#only_tr_tags = SoupStrainer("tr")
#soup = BeautifulSoup(source, 'lxml', parse_only=only_tr_tags)

# Scrape webpage into BeautifulSoup4 object

soup = BeautifulSoup(source, 'lxml')

In [7]:
# View page html data now in soup

print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],

In [4]:
# Pull wiki webpage table data with the Toronto postal code data

table_data = soup.find('table', class_= 'wikitable sortable')
#table_data

In [5]:
# Pull header data for the wiki table

headers =  table_data.find('tr')
#headers

In [8]:
#check data properties

type(headers)

bs4.element.Tag

In [6]:
# extract header text to use for column headers

column_headers = [th.getText() for th in 
                  table_data.find('tr').findAll('th')]
column_headers

['Postcode', 'Borough', 'Neighbourhood\n']

In [7]:
# Rename headers
column_headers[0] = 'PostalCode'
column_headers[2] = 'Neighborhood'
#column_headers

In [8]:
# Set table row data

rows = table_data.findAll('tr')[1:]  # skip the first header row

In [12]:
# Check data properties

type(rows)

list

In [9]:
postal_data = []  # create an empty list

for i in range(len(rows)):  # iterate table rows
    postal_row = []  # create an empty list per postal code

    # for each table data element from each table row
    for td in rows[i].findAll('td'):        
        # get the text content and append to postal_row 
        postal_row.append(td.getText())        

    # then append each postal code to the postal_data matrix
    postal_data.append(postal_row)
#postal_data

In [10]:
#Combine lists into Pandas dataframe

postal_df = pd.DataFrame(postal_data, columns = column_headers)
postal_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [11]:
 # Drop rows by condition where Borough is 'Not assigned'
    
postal_df.drop(postal_df[postal_df.Borough == 'Not assigned'].index, inplace=True)
#postal_df.head()

In [12]:
# Edit '\n' suffix from Neighborhood column elements

postal_df['Neighborhood'] = postal_df['Neighborhood'].str[:-1]
#postal_df.head()

In [13]:
# Reset index

postal_df = postal_df.reset_index(drop=True)
#postal_df.head(10)

In [14]:
# Merge / combine rows based on Postal Code, adding additional neighborhood elements, separated by comma, to respective column.

postal_df = postal_df.groupby('PostalCode').agg({'Borough':'first', 'Neighborhood': ', '.join}).reset_index() 

#change column order                           
postal_df[['PostalCode','Borough','Neighborhood']].head(3)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"


In [22]:
# Find rows with 'Neighborhood' == 'Not assigned'

postal_df[postal_df['Neighborhood'].str.match('Not assigned')]

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [15]:
# Replace 'Neighborhood' rows == 'Not Assigned' to respective row Borough name.

postal_df.Neighborhood.replace('Not assigned',postal_df.Borough,inplace=True)

In [16]:
#postal_df.head(10)

In [17]:
postal_df.shape

(103, 3)

In [17]:
# Install geocoder module.  Conda install-line needs to be run 1st time of the day for the notebook.
!conda install -c conda-forge geocoder
import geocoder

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py35_0   conda-forge
    ratelim:    0.1.6-py35_0 conda-forge

orderedset-2.0 100% |################################| Time: 0:00:00  48.50 MB/s
ratelim-0.1.6- 100% |################################| Time: 0:00:00  13.28 MB/s
geocoder-1.38. 100% |################################| Time: 0:00:00  47.24 MB/s


In [18]:
# Add column to postal_df dataframe with column elements equal to 'None'

postal_df['lat_lng_coords'] = None
postal_df.head(3)

,PostalCode,Neighborhood,Borough,lat_lng_coords
0,M1B,"Rouge, Malvern",Scarborough,None
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,None
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,None


In [19]:
# iter through the Pandas dataframe pulling geocode latitude-longitude per postal code.
# arcgis geocoder module worked very well - mostly on 1 run taking 30-40 seconds.
# Had to run it a second time once to complete the dataframe.  The recommended while loop helped a lot in this case.

for i in range(0,len(postal_df)):
    while (postal_df.lat_lng_coords[i] is None):
        postal_df.lat_lng_coords[i] = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_df.PostalCode[i])).latlng

In [20]:
# Double-check the end of the dataframe is geocoded.

postal_df.tail()

,PostalCode,Neighborhood,Borough,lat_lng_coords
98,M9N,Weston,York,"[43.70490500000005, -79.51771232799996]"
99,M9P,Westmount,Etobicoke,"[43.696300000000065, -79.53041120099994]"
100,M9R,"Kingsview Village, Martin Grove Gardens, Richv...",Etobicoke,"[43.68681000000004, -79.55728354099995]"
101,M9V,"Albion Gardens, Beaumond Heights, Humbergate, ...",Etobicoke,"[43.743795000000034, -79.58543142099995]"
102,M9W,Northwest,Etobicoke,"[43.71174000000008, -79.57940980999996]"


In [21]:
# Split the lat_lon_coords to 2 new columns in the same dataframe

postal_df[['Latitude', 'Longitude']] = pd.DataFrame(postal_df.lat_lng_coords.values.tolist(), index = postal_df.index)
postal_df.head()

,PostalCode,Neighborhood,Borough,lat_lng_coords,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,"[43.81165000000004, -79.19556138899998]",43.811650,-79.195561
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,"[43.78560500000003, -79.15870110299994]",43.785605,-79.158701
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,"[43.76569000000006, -79.17529924399997]",43.765690,-79.175299
3,M1G,Woburn,Scarborough,"[43.76821641200007, -79.21760999999998]",43.768216,-79.217610
4,M1H,Cedarbrae,Scarborough,"[43.769608066000046, -79.23943999999995]",43.769608,-79.239440


In [22]:
# Drop the lat_lng_coords column for final dataframe

postal_df.drop('lat_lng_coords', axis = 1, inplace = True)
postal_df.head()

,PostalCode,Neighborhood,Borough,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,43.811650,-79.195561
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,43.785605,-79.158701
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.765690,-79.175299
3,M1G,Woburn,Scarborough,43.768216,-79.217610
4,M1H,Cedarbrae,Scarborough,43.769608,-79.239440


In [23]:
import numpy as np # library to handle data in a vectorized manner
#import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed import folium # map rendering library
import folium # map rendering library
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  54.08 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  37.21 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  38.31 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  46.98 MB/s
Libraries imported.


In [25]:
toronto_data = postal_df[postal_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head(3)

,PostalCode,Neighborhood,Borough,Latitude,Longitude
0,M4W,Rosedale,Downtown Toronto,43.681960,-79.378445
1,M4X,"Cabbagetown, St. James Town",Downtown Toronto,43.668155,-79.366600
2,M4Y,Church and Wellesley,Downtown Toronto,43.666585,-79.381302


In [26]:
# Get the geographic coordinates for Toronto

address = 'Toronto, Ontario'
geolocator = geocoder.arcgis('Toronto, Ontario').latlng
latitude = geolocator[0]
longitude = geolocator[1]
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.648690000000045, -79.38543999999996.


In [30]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='grey',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [31]:
# The code was removed by Watson Studio for sharing.

In [32]:
# To explore a neighborhood in the Downtown Toronto borough, get it's geographic data ready to pass to Foursquare

neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood(s) name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.68196000000006, -79.37844455599998.


In [33]:
# Get the top 100 venues that are in Rosedale within a radius of 500 meters.

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=BF0XAFGTVULX1NEUNNSQBXRJ15GGGDRJ2ALMMW0PWPEIINQY&client_secret=GMRQ4FAK5IG0OIITG3EYTNNAQULI3ECDWX1V0G0XHXYGK42N&v=20180605&ll=43.68196000000006,-79.37844455599998&radius=500&limit=100'

In [35]:
# Send the GET request and check the results

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c55ec7a6a60717af116ce87'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4d6d1a35cf7e41bd9ec08285-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/financial_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d10a951735',
         'name': 'Bank',
         'pluralName': 'Banks',
         'primary': True,
         'shortName': 'Bank'}],
       'id': '4d6d1a35cf7e41bd9ec08285',
       'location': {'address': '408 Summerhill Avenue',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 484,
        'formattedAddress': ['408 Summerhill Avenue',
         'Toronto ON M4W 2E4',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.68609399737299,
  

In [36]:
# From the Foursquare lab in the previous module, we know that all the information is in the items key. 
# Borrow the get_category_type function from the Foursquare lab.

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
# Now we are ready to clean the json and structure it into a pandas dataframe.

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(3)

,name,categories,lat,lng
0,TD Canada Trust - Closed,Bank,43.686094,-79.376549
1,Rosedale Park,Playground,43.682328,-79.378934
2,Rosedale Tennis Club,Tennis Court,43.683226,-79.378984


In [39]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Downtown Toronto
#### Let's replicate the function to repeat the same process to all the neighborhoods in Downtown Toronto

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [41]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


In [42]:
print(toronto_venues.shape)
toronto_venues.head()

(1234, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.68196,-79.378445,TD Canada Trust - Closed,43.686094,-79.376549,Bank
1,Rosedale,43.68196,-79.378445,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.68196,-79.378445,Rosedale Tennis Club,43.683226,-79.378984,Tennis Court
3,Rosedale,43.68196,-79.378445,Whitney Park,43.682036,-79.373788,Park
4,Rosedale,43.68196,-79.378445,Mooredale House,43.678631,-79.380091,Building


In [43]:
#Let's check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,63,63,63,63,63,63
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",66,66,66,66,66,66
"Cabbagetown, St. James Town",42,42,42,42,42,42
Central Bay Street,97,97,97,97,97,97
"Chinatown, Grange Park, Kensington Market",93,93,93,93,93,93
Christie,9,9,9,9,9,9
Church and Wellesley,82,82,82,82,82,82
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [44]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 180 uniques categories.


## 3. Analyze Each Neighborhood

In [45]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(3)

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sports Bar,Steakhouse,Strip Club,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
# Check the new dataframe size.

toronto_onehot.shape

(1234, 180)

#### Next, group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [47]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sports Bar,Steakhouse,Strip Club,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.040000,0.000000,0.010000,0.000000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.030000,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.000000,0.020000,0.01,0.020000,0.000000,0.01,0.020000,0.010000,0.00000,0.060000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.050000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.010000,0.000000,0.00000,0.03000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.00000,0.000000,0.01,0.01,0.000000,0.010000,0.000000,0.020000,0.010000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.070000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.030000,0.000000,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.010000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.01,0.000000,0.00,0.00000,0.010000,0.00,0.000000,0.00000,0.000000,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,0.020000,0.010000,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.00,0.000000,0.01,0.000000,0.010000,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.00000,0.000000,0.000000,0.010000,0.0,0.020000,0.010

#### Let's confirm the new size

In [48]:
toronto_grouped.shape

(18, 180)

#### Let's print each neighborhood along with the top 5 most common venues

In [49]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0                Hotel  0.07
1                 Café  0.06
2          Coffee Shop  0.05
3  American Restaurant  0.04
4           Steakhouse  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1    Restaurant  0.06
2  Cocktail Bar  0.05
3           Pub  0.03
4   Cheese Shop  0.03


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                  venue  freq
0           Coffee Shop  0.09
1    Italian Restaurant  0.06
2            Restaurant  0.05
3  Gym / Fitness Center  0.05
4                  Café  0.05


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.10
1          Restaurant  0.07
2                Park  0.05
3  Chinese Restaurant  0.05
4              Bakery  0.05


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.11
1  Clothing Store  0.08
2           Plaza  0.03
3        Te

#### Let's put that into a *pandas* dataframe

In [50]:
# First, let's use the function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Hotel,Café,Coffee Shop,American Restaurant,Gastropub,Asian Restaurant,Bakery,Bar,Steakhouse,Deli / Bodega
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Hotel,Seafood Restaurant,Farmers Market,Steakhouse,Pub,Italian Restaurant,Café
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Pub,Restaurant,Diner,Park,Sandwich Place,Bar
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Bakery,Park,Café,Chinese Restaurant,Sandwich Place,Butcher
4,Central Bay Street,Coffee Shop,Clothing Store,Café,Tea Room,Plaza,Restaurant,Pizza Place,Chinese Restaurant,Bar,Spa


## 4. Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 5 clusters.

In [52]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 3, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [53]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Rosedale,Downtown Toronto,43.681960,-79.378445,2,Tennis Court,Building,Playground,Park,Bank,Electronics Store,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,M4X,"Cabbagetown, St. James Town",Downtown Toronto,43.668155,-79.366600,1,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Bakery,Park,Café,Chinese Restaurant,Sandwich Place,Butcher
2,M4Y,Church and Wellesley,Downtown Toronto,43.666585,-79.381302,1,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Burger Joint,Sushi Restaurant,Pub,Pizza Place,Dance Studio,Gastropub
3,M5A,"Harbourfront, Regent Park",Downtown Toronto,43.655120,-79.362640,4,Coffee Shop,Restaurant,Breakfast Spot,Thai Restaurant,Electronics Store,Event Space,Bakery,Pub,Mexican Restaurant,Italian Restaurant
4,M5B,"Ryerson, Garden District",Downtown Toronto,43.657363,-79.378180,1,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Ramen Restaurant,Plaza,Lingerie Store,Japanese Restaurant,Italian Restaurant


Finally, let's visualize the resulting clusters

In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,"Harbourfront East, Toronto Islands, Union Station",0,Harbor / Marina,Park,Music Venue,Athletics & Sports,Women's Store,Electronics Store,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


#### Cluster 2

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Cabbagetown, St. James Town",1,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Bakery,Park,Café,Chinese Restaurant,Sandwich Place,Butcher
2,Church and Wellesley,1,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Burger Joint,Sushi Restaurant,Pub,Pizza Place,Dance Studio,Gastropub
4,"Ryerson, Garden District",1,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Ramen Restaurant,Plaza,Lingerie Store,Japanese Restaurant,Italian Restaurant
5,St. James Town,1,Coffee Shop,Restaurant,Hotel,Café,Bakery,Breakfast Spot,Cocktail Bar,Clothing Store,Gastropub,Cosmetics Shop
6,Berczy Park,1,Coffee Shop,Restaurant,Cocktail Bar,Hotel,Seafood Restaurant,Farmers Market,Steakhouse,Pub,Italian Restaurant,Café
7,Central Bay Street,1,Coffee Shop,Clothing Store,Café,Tea Room,Plaza,Restaurant,Pizza Place,Chinese Restaurant,Bar,Spa
8,"Adelaide, King, Richmond",1,Hotel,Café,Coffee Shop,American Restaurant,Gastropub,Asian Restaurant,Bakery,Bar,Steakhouse,Deli / Bodega
10,"Design Exchange, Toronto Dominion Centre",1,Coffee Shop,Café,Hotel,American Restaurant,Steakhouse,Restaurant,Japanese Restaurant,Deli / Bodega,Bar,Asian Restaurant
11,"Commerce Court, Victoria Hotel",1,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Gastropub,Deli / Bodega,Seafood Restaurant,Japanese Restaurant
12,"Harbord, University of Toronto",1,Café,Coffee Shop,Restaurant,Bakery,Gym,Greek Restaurant,Bookstore,Japanese Restaurant,Thai Restaurant,Bar


#### Cluster 3

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rosedale,2,Tennis Court,Building,Playground,Park,Bank,Electronics Store,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


#### Cluster 4

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Christie,3,Café,Grocery Store,Baby Store,Playground,Italian Restaurant,Coffee Shop,Electronics Store,Fast Food Restaurant,Farmers Market,Farm


#### Cluster 5

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Harbourfront, Regent Park",4,Coffee Shop,Restaurant,Breakfast Spot,Thai Restaurant,Electronics Store,Event Space,Bakery,Pub,Mexican Restaurant,Italian Restaurant
